In [ ]:
!pip install mediapipe
import mediapipe as mp
import cv2 
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import clear_output
os.chdir(r'/kaggle/input/yoga-pose-classification/YogaPoses')
clear_output()

In [ ]:
labels=os.listdir('.')
labels

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
images=[]
for idx,data in enumerate(os.listdir('.')):
    for image in os.listdir(f'{data}'):
        image=cv2.imread(f'{data}/{image}')
        # or
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        try :
            landmarks = results.pose_landmarks

            for landmark in landmarks.landmark:
               x = landmark.x
               y = landmark.y

               # Do something with the coordinates
               mp_drawing = mp.solutions.drawing_utils
               mp_drawing.draw_landmarks(image_rgb, landmarks, mp_pose.POSE_CONNECTIONS)
               images.append(image_rgb)
               break
        except:
            continue
        break
    if idx==4:
        break
fig,axes=plt.subplots(2,2,figsize=(8,7))  

for i ,ax in enumerate(axes.flat) :
    ax.imshow(images[i])
    ax.set_title(labels[i],fontsize=8)

plt.show()

In [ ]:
import pandas as pd

# Create a dictionary with data
data_set =defaultdict(list)

# Create a DataFrame from the dictionary
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
label_encode=[]
for idx,data in enumerate(os.listdir('.')):
    for image in os.listdir(f'{data}'):
        img=cv2.imread(f'{data}/{image}')

        image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        landmarks = results.pose_landmarks
        try:
            for idd,landmark in enumerate(landmarks.landmark):
               x = landmark.x
               y = landmark.y
               
               data_set[f'X {idd}']=(data_set[f'X {idd}']+[x])
               data_set[f'Y {idd}']=(data_set[f'Y {idd}']+[y])
            data_set['Labels']=(data_set['Labels']+[idx]) 
            data_set['image_Name']=(data_set['image_Name']+[image])
        except:
            pass

In [ ]:
data=pd.DataFrame(data_set)
data

In [ ]:
data.Labels.value_counts()

In [ ]:
from  sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
target=data['Labels']
feutures=data.drop('Labels',axis=1)
X_train, X_test, y_train, y_test = train_test_split(feutures, target, test_size=0.2, random_state=42)


In [ ]:
analysis_error=X_test['image_Name']
X_train.drop('image_Name',inplace=True,axis=1)
X_test.drop('image_Name',inplace=True,axis=1)

In [ ]:
analysis_error

In [ ]:
print('X_train shape is',X_train.shape )
print('X_test shape is',X_test.shape )
print('y_train shape is',y_train.shape )
print('y_test shape is',y_test.shape )
print('analysis_error shape is',analysis_error.shape )

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=120, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import numpy as np
np.sum(y_pred!=y_test)

In [ ]:
images=[]
false_label=y_pred[y_pred!=y_test]
y_=np.array(y_test)
true_label=y_[y_pred!=y_test]
for i in analysis_error[(y_pred!=y_test)].index:
    image_path=labels[y_test[i]]+'/'+analysis_error[i]
    image=plt.imread(image_path)
    images.append(image)

fig,axes=plt.subplots(2,7,figsize=(10,5))  

for i ,ax in enumerate(axes.flat) :
    ax.imshow(images[i])
    ax.set_title(f'{false_label[i]}-->{true_label[i]}',fontsize=8)
    ax.set_xticks([])  # Remove tick labels from x-axis
    ax.set_yticks([])  
 
plt.show()

In [ ]:
#Here you often find images that are misclassified because mislabel in data 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming you have the true labels (y_true) and predicted labels (y_pred)
cm = confusion_matrix(y_test, y_pred)

# Create a heatmap of the confusion matrix
sns.heatmap(cm, annot=True, cmap='Blues')

# Set the axis labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Show the plot
plt.show()